In [44]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#reading in CSV, assigning to 'stock_data' and checking dataframe

stock_data = pd.read_csv('./stock_data/stocks_latest/stock_prices_latest.csv')
stock_data.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,49.7013,20032017,1.0
1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,22.5902,30977700,1.0
2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,18.0802,41591300,1.0
3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,27.2232,288121200,1.0
4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,38.6773,74640000,1.0


In [3]:
#Converting date to datetime then filtering down just to stock prices from 2019 on

stock_data['date'] = pd.to_datetime(stock_data['date'])
stock_data_trim = stock_data[(stock_data['date'] > '2018-12-31')]
stock_data_trim.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
18056482,MXWL,2019-01-22,2.89,3.19,2.80,3.13,3.13,684721,1.0
18056486,MXWL,2019-01-10,2.35,2.37,2.25,2.33,2.33,162318,1.0
18056488,MXWL,2019-01-09,2.48,2.54,2.27,2.30,2.30,203517,1.0
18056491,MXWL,2019-01-17,2.84,2.90,2.66,2.86,2.86,504809,1.0
18056499,MXWL,2019-01-25,2.91,2.98,2.82,2.85,2.85,322918,1.0


In [4]:
#confirming that the new dataframe is smaller than the old
print(stock_data.shape)
print(stock_data_trim.shape)

(23528435, 9)
(2985111, 9)


In [20]:
#Sorting stock data by symbol and date
stock_data_trim = stock_data_trim.sort_values(['symbol', 'date'])
stock_data_trim.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient,lag,return
18289679,A,2019-01-02,66.50,66.570,65.30,65.69,65.69,2113304,1.0,63.27,-0.036840
18289678,A,2019-01-03,65.53,65.780,62.00,63.27,63.27,5383926,1.0,2.59,-0.959064
18289691,A,2019-01-04,64.09,65.950,64.09,65.46,65.46,3123654,1.0,70.38,0.075160
18289717,A,2019-01-07,65.64,67.425,65.61,66.85,66.85,3235055,1.0,72.18,0.079731
18289707,A,2019-01-08,67.59,68.210,66.70,67.83,67.83,1578055,1.0,71.32,0.051452


In [64]:
#Creating new columns for lag and return then populating them based on the previous day's stock data
stock_data_trim['lag'] = ""
stock_data_trim['return'] = ""

#will want to think about better ways to impute missing here
stock_data_trim.lag = np.where(stock_data_trim.symbol.eq(stock_data_trim.symbol.shift()), stock_data_trim['close'].shift(1), 9999999)
stock_data_trim['return'] = np.where(stock_data_trim.symbol.eq(stock_data_trim.symbol.shift()), ((stock_data_trim['lag'] / stock_data_trim['close']) - 1), 999999)

#checking a stock to make sure it is working as expected
stock_data_trim.loc[stock_data_trim['symbol'] == 'ALGN'].head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient,lag,return
18328176,ALGN,2019-01-02,205.32,207.16,200.540,202.12,202.12,1126789.0,1.0,9999999.00,999999.000000
18328212,ALGN,2019-01-03,200.03,200.77,183.620,184.78,184.78,2414312.0,1.0,202.12,0.093841
18328189,ALGN,2019-01-04,181.68,190.39,177.925,186.71,186.71,2518182.0,1.0,184.78,-0.010337
18328261,ALGN,2019-01-07,187.41,190.77,184.505,189.92,189.92,1311217.0,1.0,186.71,-0.016902
18328201,ALGN,2019-01-08,191.35,194.99,187.590,192.95,192.95,1001499.0,1.0,189.92,-0.015704


In [5]:
#reading in company descriptions, assigning to 'company_data' and checking dataframe
company_data = pd.read_csv('./S&P500-Info_Twitter_link.csv')

#dropping unnamed column and checking dataframe
company_data = company_data.drop(['Unnamed: 0'], axis =1)
company_data.head()

,symbol,security,sec filings,gics sector,gics sub-industry,headquarters location,date first added,cik,founded,twitter handle
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902,3mnews
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888,abbottglobal
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888),abbvieus
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981,abiomedimpella
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989,accenture


In [6]:
#merging stock and company description dataframes, only keeping S&P 500 company stock data
company_stock_data = stock_data_trim.merge(company_data, on = 'symbol')

#checking to make sure that all others have been removed
company_stock_data['security'].isna().sum()

0

In [7]:
company_stock_data.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient,security,sec filings,gics sector,gics sub-industry,headquarters location,date first added,cik,founded,twitter handle
0,CXO,2019-01-15,122.76,123.94,121.860,122.43,122.43,1405339,1.0,Concho Resources,reports,Energy,Oil & Gas Exploration & Production,"Midland, Texas",2/22/2016,1358071,2004,conchooilfield
1,CXO,2019-01-11,120.66,121.78,119.350,120.45,120.45,1369295,1.0,Concho Resources,reports,Energy,Oil & Gas Exploration & Production,"Midland, Texas",2/22/2016,1358071,2004,conchooilfield
2,CXO,2019-01-03,107.12,108.98,105.260,106.86,106.86,1564283,1.0,Concho Resources,reports,Energy,Oil & Gas Exploration & Production,"Midland, Texas",2/22/2016,1358071,2004,conchooilfield
3,CXO,2019-01-08,116.68,120.10,116.675,119.45,119.45,2664825,1.0,Concho Resources,reports,Energy,Oil & Gas Exploration & Production,"Midland, Texas",2/22/2016,1358071,2004,conchooilfield
4,CXO,2019-01-02,100.42,106.98,100.140,106.69,106.69,1640711,1.0,Concho Resources,reports,Energy,Oil & Gas Exploration & Production,"Midland, Texas",2/22/2016,1358071,2004,conchooilfield


In [37]:
# #Exporting trimmed file
# company_stock_data.to_csv(r'\stock_prices_trimmed.csv', index = False)